In [1]:
%load_ext autoreload
import os
import sys

In [2]:
os.environ['TRANSFORMERS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache-new'
os.environ['HF_DATASETS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache-new'
os.environ["OPENAI_API_KEY"]= 'xxx'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
sys.path.append('./src-py')

In [3]:
import json
import os
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import re
from collections import Counter

pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


%autoreload
import utils
import prompts
import random

from tabulate import tabulate
import tiktoken
from llm_based_evaluation import *

/mnt/swordfish-pool2/milad/conda-envs/trl-library-env/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
from transformers import AutoTokenizer, pipeline

In [5]:
import json
keys = json.load(open('./keys.json'))
for key, val in keys.items():
    os.environ[key] = val

In [6]:
from huggingface_hub import login

login(os.environ['hf_token'])

In [7]:
output_dir = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/'
models_folder = "/mnt/swordfish-pool2/milad/communicating-science-to-the-public/models/"

In [8]:
gpt_tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

In [9]:
ds_path = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/new-eval_experiment_500/'
#ds_path = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/new-eval_experiment_100'

In [10]:
# Converting the ground-truth dataset so similar fomrat as the generated ones
# gt_ds = datasets.load_from_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/new-deepseek-final-conv-ds-cleaned/')
# gen_dataset = datasets.load_from_disk(ds_path + '/baseline_qwen_gen_conv_fixed_researcher_simple_prompt')
# pr_titles = gen_dataset['paper_title']

# gt_ds = gt_ds.filter(lambda row: row['pr-title'] in pr_titles)
# gt_ds = gt_ds.map(lambda row: {'conversation': "\n\n".join(["{}:{}".format(turn['author'], turn['text']) for turn in row['parsed_conv']])})
# gt_ds.save_to_disk(ds_path + '/ground_truth_conversation_ds')
# gt_ds.push_to_hub('miladalsh/ground-truth-conv-500')

### Evaluating the Researcher

#### Using DeepSeek

In [28]:
generated_convs = {
    'qwen-baseline':[datasets.load_from_disk(ds_path + '/baseline_qwen_gen_conv_fixed_quesions/'), ds_path + '/baseline_qwen_gen_conv_fixed_quesions'],
    'ft-qwen-on-deepseek' :[datasets.load_from_disk(ds_path + '/ft_qwen_gen_conv_fixed_quesions/'),ds_path +  '/ft_qwen_gen_conv_fixed_quesions/'],
    'llama-baseline':[datasets.load_from_disk(ds_path + '/baseline_llama3_gen_conv_fixed_quesions/'), ds_path + '/baseline_llama3_gen_conv_fixed_quesions'],
    'ft-llama-on-deepseek' :[datasets.load_from_disk(ds_path + '/ft_llama3_gen_conv_fixed_quesions/'),ds_path +  '/ft_llama3_gen_conv_fixed_quesions/'],
}

In [13]:
#print(generated_convs['ft-qwen-on-deepseek'][0]['fixed_journalist_conv'][0])

In [ ]:
prompts_to_eval = [prompts.clarity_eval_prompt, prompts.scientific_context_eval_prompt, prompts.societal_context_eval_prompt]
for p in prompts_to_eval:
    p['inputs']['Conversation'] = 'fixed_journalist_conv'

llm_eval_results = llm_based_evaluation(prompts_to_eval, generated_convs, force_generation=False, evaluator_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-14B")

100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [37:52<00:00,  4.55s/it]


Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

  2%|█▌                                                                              | 10/500 [00:41<33:16,  4.08s/it]

In [41]:
llm_eval_results = llm_based_evaluation(prompts_to_eval, generated_convs, force_generation=False, evaluator_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-14B")

Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/new-eval_experiment_500/baseline_qwen_gen_conv_fixed_quesions/ds_eval/deepseek-ai from already saved file
Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/new-eval_experiment_500/ft_qwen_gen_conv_fixed_quesions//ds_eval/deepseek-ai from already saved file
Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/new-eval_experiment_500/baseline_llama3_gen_conv_fixed_quesions/ds_eval/deepseek-ai from already saved file
Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/new-eval_experiment_500/ft_llama3_gen_conv_fixed_quesions//ds_eval/deepseek-ai from already saved file
#                       clarity_eval_prompt    scientific_eval_prompt    societal_eval_prompt    Avg
--------------------  ---------------------  ------------------------  ----------------------  -----
qwen-baseline                          3.75                      2.97                    2.91   

### Evaluating the Journalist

#### Basic Eval

In [11]:
generated_convs = {
    'qwen-baseline-simple':datasets.load_from_disk(ds_path + '/baseline_qwen_gen_conv_fixed_researcher_simple_prompt'),
    'qwen-baseline':datasets.load_from_disk(ds_path + '/baseline_qwen_gen_conv_fixed_researcher'),
    'ft-qwen-on-deepseek-simple-prompt' :datasets.load_from_disk(ds_path + '/ft_qwen_gen_conv_fixed_researcher'),
    'dpo-qwen' : datasets.load_from_disk(ds_path + '/dpo_qwen_gen_conv_fixed_researcher'),
    'llama-baseline-simple':datasets.load_from_disk(ds_path + '/baseline_llama3_gen_conv_fixed_researcher_simple_prompt'),
    'llama-baseline':datasets.load_from_disk(ds_path + '/baseline_llama3_gen_conv_fixed_researcher'),
    'ft-llama-on-deepseek-simple-prompt' :datasets.load_from_disk(ds_path + '/ft_llama3_gen_conv_fixed_researcher'),
    'dpo-llama' : datasets.load_from_disk(ds_path + '/dpo_llama3_gen_conv_fixed_researcher')
}

In [13]:
#print("{}\n---\n\n".format(generated_convs['ft-llama-on-deepseek']['paper_title'][4]),"\n\n==\n\n".join([x['content'] for x in generated_convs['ft-llama-on-deepseek']['generated_conversation'][4] if x['role'] == 'assistant']))

In [12]:
print('Results on 500 sample')
all_synth_conversations = utils.evalaute_convs(generated_convs)

Results on 500 sample


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Prompt                                Rouge-1    Rouge-L    BERT-f1
----------------------------------  ---------  ---------  ---------
qwen-baseline-simple                    0.33       0.14       0.812
qwen-baseline                           0.314      0.135      0.818
ft-qwen-on-deepseek-simple-prompt       0.395      0.154      0.826
dpo-qwen                                0.36       0.143      0.822
llama-baseline-simple                   0.345      0.148      0.821
llama-baseline                          0.339      0.146      0.816
ft-llama-on-deepseek-simple-prompt      0.398      0.16       0.827
dpo-llama                               0.348      0.147      0.82


#### LLM-based Eval

In [14]:
prompts_to_eval = [prompts.journalist_clarity_eval_prompt, prompts.journalist_scientific_context_eval_prompt, prompts.journalist_societal_context_eval_prompt]

In [59]:
generated_convs = {
    'qwen-baseline-simple':[datasets.load_from_disk(ds_path + '/baseline_qwen_gen_conv_fixed_researcher_simple_prompt'), ds_path + '/baseline_qwen_gen_conv_fixed_researcher_simple_prompt'],
    'qwen-baseline':[datasets.load_from_disk(ds_path + '/baseline_qwen_gen_conv_fixed_researcher'), ds_path + '/baseline_qwen_gen_conv_fixed_researcher'],
    'ft-qwen-on-deepseek-simple-prompt' :[datasets.load_from_disk(ds_path + '/ft_qwen_gen_conv_fixed_researcher'), ds_path + '/ft_qwen_gen_conv_fixed_researcher'],
    'dpo-qwen' : [datasets.load_from_disk(ds_path + '/dpo_qwen_gen_conv_fixed_researcher'),ds_path + '/dpo_qwen_gen_conv_fixed_researcher'],
    'llama-baseline-simple':[datasets.load_from_disk(ds_path + '/baseline_llama3_gen_conv_fixed_researcher_simple_prompt'),ds_path + '/baseline_llama3_gen_conv_fixed_researcher_simple_prompt'],
    'llama-baseline':[datasets.load_from_disk(ds_path + '/baseline_llama3_gen_conv_fixed_researcher'),ds_path + '/baseline_llama3_gen_conv_fixed_researcher'],
    'ft-llama-on-deepseek-simple-prompt' :[datasets.load_from_disk(ds_path + '/ft_llama3_gen_conv_fixed_researcher'),ds_path + '/ft_llama3_gen_conv_fixed_researcher'],
    'dpo-llama' : [datasets.load_from_disk(ds_path + '/dpo_llama3_gen_conv_fixed_researcher'),ds_path + '/dpo_llama3_gen_conv_fixed_researcher'],
}

In [16]:
# Uploading conversations into github
for key,val in generated_convs.items():
    val[0].push_to_hub('miladalsh/gen-conv-by-{}'.format(key))
    # Upload the eval_results
    eval_res = datasets.load_from_disk(val[1] + '/ds_journalist_eval/deepseek-ai/')
    eval_res.push_to_hub('miladalsh/gen-conv-by-{}-with-eval'.format(key))

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/12.4M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

No files have been modified since last commit. Skipping to prevent empty commit.


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


In [17]:
generated_convs['original_deepseek_conv']    = [datasets.load_from_disk(ds_path + '/ground_truth_conversation_ds'), ds_path + '/ground_truth_conversation_ds']
generated_convs['original_deepseek_conv'][0] = generated_convs['original_deepseek_conv'][0].remove_columns(['scientific_eval_prompt_scoring_parsed', 'clarity_eval_prompt_scoring_parsed', 'societal_eval_prompt_scoring_parsed'])
generated_convs['original_deepseek_conv'][0] = generated_convs['original_deepseek_conv'][0].map(lambda row: {'generated_conversation': [{'role': 'assistant' if 'journalist' in utter['author'].lower() else 'user', 
                                                                                                                                        'content':utter['text']
                                                                                                                                       } for utter in row['parsed_conv']]})

# generated_convs['original_deepseek_conv_2']    = [datasets.load_from_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-32-final-conv-1k-sample/'), '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-32-final-conv-ds-test/']
# generated_convs['original_deepseek_conv_2'][0] = generated_convs['original_deepseek_conv_2'][0].remove_columns(['scientific_eval_prompt_scoring_parsed', 'clarity_eval_prompt_scoring_parsed', 'societal_eval_prompt_scoring_parsed'])
# generated_convs['original_deepseek_conv_2'][0] = generated_convs['original_deepseek_conv_2'][0].map(lambda row: {'generated_conversation': [{'role': 'assistant' if 'journalist' in utter['author'].lower() else 'user', 
#                                                                                                                                         'content':utter['text']
#                                                                                                                                        } for utter in row['parsed_conv']]})

In [18]:
llm_eval_results = llm_based_evaluation(prompts_to_eval, generated_convs, force_generation=False, evaluator_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-32B", eval_conv_questions=True)

Loading deepseek-ai from already saved file
Loading deepseek-ai from already saved file
Loading deepseek-ai from already saved file
Loading deepseek-ai from already saved file
Loading deepseek-ai from already saved file
Loading deepseek-ai from already saved file
Loading deepseek-ai from already saved file
Loading deepseek-ai from already saved file
Loading deepseek-ai from already saved file
#                                     journalist_clarity_eval_prompt    journalist_scientific_context_eval_prompt    journalist_societal_context_eval_prompt    Avg
----------------------------------  --------------------------------  -------------------------------------------  -----------------------------------------  -----
qwen-baseline-simple                                            0.64                                         0.41                                       0.07   0.16
qwen-baseline                                                   0.69                                         0.4

In [35]:
llm_eval_results['dpo-llama']['generated_conversation'][0][2:]

[{'content': 'Could you clarify how the study defines "vegetarian" and "pescetarian," and how these definitions align with broader cultural or societal understandings of these terms?',
  'role': 'assistant'},
 {'content': 'In this study, the researchers defined "vegetarian" as individuals who do not eat meat, fish, dairy products, or eggs. This means that vegans, who do not consume any animal products, are included in the vegetarian group. The study also includes individuals who do not eat meat but do consume dairy products and eggs, which is often referred to as a "lacto-ovo-vegetarian" diet.\n\nOn the other hand, the study defines "pescetarian" as individuals who consume fish but do not eat meat. This means that pescetarians may also consume dairy products and eggs, but they do not eat meat from land animals.\n\nIt\'s worth noting that these definitions may not align with broader cultural or societal understandings of these terms. In some cultures, "vegetarian" may refer specifically

In [17]:
#llm_eval_results['ft-llama-on-deepseek-simple-prompt']['generated_conversation'][4][2:]

In [18]:
#llm_eval_results['ft-qwen-on-deepseek-simple-prompt']['generated_conversation'][4][2:]

In [51]:
#len(set(llm_eval_results['original_deepseek_conv']['pr-title']).intersection(set(llm_eval_results['ft-llama-on-deepseek-simple-prompt']['paper_title'])))

In [19]:
#llm_eval_results['original_deepseek_conv']['generated_conversation'][1][2:]

In [26]:
#print(llm_eval_results['dpo-llama']['journalist_scientific_context_eval_prompt_scoring_parsed'][1])

{'reasons': {'all_questions': ['Could you clarify how exactly the entanglement between the two atomic ensembles was established over such a long distance?', 'Specifically, how did you ensure that the entanglement was not lost during transmission, and what role did the cavity enhancement and quantum frequency conversion play in this process?', 'Could you elaborate on how the QFC module specifically ensured the entanglement was preserved during transmission, and what exactly happened during the frequency conversion process that maintained the entanglement?', 'Additionally, how did you ensure that the entanglement was not lost due to the long distance or potential noise introduced during the conversion?', 'Could you provide more details on how the spatial and spectral properties of the entangled photon are preserved during the frequency conversion, and what specific mechanisms ensure that the entanglement is not lost or degraded during this process?', "Could you explain how the PPLN-WG en

In [22]:
#llm_eval_results['ft-qwen-on-deepseek-simple-prompt']['generated_conversation'][7][2:]

In [80]:
# quanitfy how often the journalist switches role (doesn't ask questions)
data = []
for key, ds in generated_convs.items():
    ds = ds[0]
    journalist_convs = [[x['content'] for x in conv if x['role'] == 'assistant'] for conv in ds['generated_conversation']]

    # Average number of questions per turn
    journalist_questions = [extract_questions(turn) for conv in journalist_convs for turn in conv]
    avg_num_ques_per_turn = round(np.mean([len(conv) for conv in journalist_questions]), 2)

    # Average number of turns with no questions 
    percent_of_turns_with_no_ques = round(len([turn for turn in journalist_questions if len(turn) == 0]) / len(journalist_questions), 2)
    
    # Average number of questions per conv
    conv_questions = [extract_questions('\n\n'.join(turns)) for turns in journalist_convs]
    avg_num_ques_per_conv = round(np.mean([len(conv) for conv in conv_questions]), 1)

    # Average number of turns with no questions
    xx = [[extract_questions(turn) for turn in conv] for conv in journalist_convs]
    
    data.append([key, percent_of_turns_with_no_ques, avg_num_ques_per_turn, avg_num_ques_per_conv])

print(tabulate(data, headers=['#', '% of turns w/o questions', '# ques/turn', '# ques/conv']))

#                                     % of turns w/o questions    # ques/turn    # ques/conv
----------------------------------  --------------------------  -------------  -------------
qwen-baseline-simple                                      0.65           1.66            8.3
qwen-baseline                                             0.29           1.46            7.3
ft-qwen-on-deepseek-simple-prompt                         0.16           1.04            5.2
dpo-qwen                                                  0.17           1.56            7.8
llama-baseline-simple                                     0.11           2.78           13.9
llama-baseline                                            0.02           2.21           11.1
ft-llama-on-deepseek-simple-prompt                        0.04           1.08            5.4
dpo-llama                                                 0.18           1.13            5.6


Performing statistical analysis on the ta scores

In [16]:
 llm_eval_results['ft-llama-on-deepseek-simple-prompt']

Dataset({
    features: ['paper_id', 'paper_title', 'paper_text', 'prompt', 'completion', 'pr-article', 'topic', '__index_level_0__', 'generated_conversation', 'conversation', 'journalist_clarity_eval_prompt_scoring_parsed', 'journalist_scientific_context_eval_prompt_scoring_parsed', 'journalist_societal_context_eval_prompt_scoring_parsed'],
    num_rows: 100
})

In [18]:
ft_scores = [x['score'] for x in llm_eval_results['ft-llama-on-deepseek-simple-prompt']['journalist_clarity_eval_prompt_scoring_parsed']]
bl_scores = [x['score'] for x in llm_eval_results['llama-baseline']['journalist_clarity_eval_prompt_scoring_parsed']]
utils.stats_analysis(ft_scores, bl_scores)
print('=====')
ft_scores = [x['score'] for x in llm_eval_results['ft-llama-on-deepseek-simple-prompt']['journalist_scientific_context_eval_prompt_scoring_parsed']]
bl_scores = [x['score'] for x in llm_eval_results['llama-baseline']['journalist_scientific_context_eval_prompt_scoring_parsed']]
utils.stats_analysis(ft_scores, bl_scores)
print('=====')
ft_scores = [x['score'] for x in llm_eval_results['ft-llama-on-deepseek-simple-prompt']['journalist_societal_context_eval_prompt_scoring_parsed']]
bl_scores = [x['score'] for x in llm_eval_results['llama-baseline']['journalist_societal_context_eval_prompt_scoring_parsed']]
utils.stats_analysis(ft_scores, bl_scores)

Paired T-test Results:
T-statistic: 2065.5
P-value: 0.6456451957848015

Interpretation:
The p-value (0.6456) is greater than the significance level (0.05).
Conclusion: There is no statistically significant difference between the two systems.
=====
Paired T-test Results:
T-statistic: 1921.0
P-value: 0.18264351548240743

Interpretation:
The p-value (0.1826) is greater than the significance level (0.05).
Conclusion: There is no statistically significant difference between the two systems.
=====
Paired T-test Results:
T-statistic: 1120.5
P-value: 0.10770328491534613

Interpretation:
The p-value (0.1077) is greater than the significance level (0.05).
Conclusion: There is no statistically significant difference between the two systems.


In [19]:
ft_scores = [x['score'] for x in llm_eval_results['ft-qwen-on-deepseek-simple-prompt']['journalist_clarity_eval_prompt_scoring_parsed']]
bl_scores = [x['score'] for x in llm_eval_results['qwen-baseline']['journalist_clarity_eval_prompt_scoring_parsed']]
utils.stats_analysis(ft_scores, bl_scores)
print('=====')
ft_scores = [x['score'] for x in llm_eval_results['ft-qwen-on-deepseek-simple-prompt']['journalist_scientific_context_eval_prompt_scoring_parsed']]
bl_scores = [x['score'] for x in llm_eval_results['qwen-baseline']['journalist_scientific_context_eval_prompt_scoring_parsed']]
utils.stats_analysis(ft_scores, bl_scores)
print('=====')
ft_scores = [x['score'] for x in llm_eval_results['ft-qwen-on-deepseek-simple-prompt']['journalist_societal_context_eval_prompt_scoring_parsed']]
bl_scores = [x['score'] for x in llm_eval_results['qwen-baseline']['journalist_societal_context_eval_prompt_scoring_parsed']]
utils.stats_analysis(ft_scores, bl_scores)

Paired T-test Results:
T-statistic: 1190.0
P-value: 0.0021805381552662357

Interpretation:
The p-value (0.0022) is less than the significance level (0.05).
Conclusion: There is a statistically significant difference between the two systems.
=====
Paired T-test Results:
T-statistic: 1824.0
P-value: 0.09051180549476667

Interpretation:
The p-value (0.0905) is greater than the significance level (0.05).
Conclusion: There is no statistically significant difference between the two systems.
=====
Paired T-test Results:
T-statistic: 1014.0
P-value: 0.003645799178622852

Interpretation:
The p-value (0.0036) is less than the significance level (0.05).
Conclusion: There is a statistically significant difference between the two systems.


-------

### Analysis of the automatic evalaution

In [26]:
# writing sample conversations to label-studio

In [21]:
output_data = []
for item in range(10):
    
    b_llama3 = llm_eval_results['llama-baseline-simple']['generated_conversation'][item][2:]
    ft_llama3 = llm_eval_results['ft-llama-on-deepseek-simple-prompt']['generated_conversation'][item][2:]
    
    b_qwen  = llm_eval_results['qwen-baseline']['generated_conversation'][item][2:]
    ft_qwen = llm_eval_results['ft-qwen-on-deepseek-simple-prompt']['generated_conversation'][item][2:]
    
    label_studio_item = {
        'paper_title': llm_eval_results['ft-llama-on-deepseek-simple-prompt'][item]['paper_title'],
        'paper_text': llm_eval_results['ft-llama-on-deepseek-simple-prompt'][item]['paper_text'],
        'b_llama3': [{'author': x['role'], 'text': x['content']} for x in b_llama3],
        'ft_llama3': [{'author': x['role'], 'text': x['content']} for x in ft_llama3],
        'b_qwen': [{'author': x['role'], 'text': x['content']} for x in b_qwen],
        'ft_qwen': [{'author': x['role'], 'text': x['content']} for x in ft_qwen]
    }
    output_data.append(label_studio_item)
json.dump(output_data, open('convs.json', 'w'))

In [52]:
print(output_data[2]['ft_qwen'])

[{'author': 'assistant', 'text': "Thank you for sharing your groundbreaking research. Let's dive into the details. First, how did you establish the link between RNA splicing and aging?"}, {'author': 'user', 'text': 'Establishing the link between RNA splicing and aging was a crucial step in our research. We used a combination of experimental and computational approaches to investigate the relationship between RNA splicing and aging in Caenorhabditis elegans, a nematode worm that is a popular model organism in aging research.\n\nFirst, we created a reporter strain that expresses a pair of ret-1 exon 5 reporter minigenes with differential frameshifts, driven by the ubiquitous eft-3 promoter. This reporter strain allowed us to monitor alternative splicing in real-time, using fluorescence microscopy. We then used this reporter to examine the splicing patterns in young and old animals, as well as in animals fed ad libitum or subjected to dietary restriction.\n\nOur results showed that, as an